In [230]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
%matplotlib inline

# import interactive shell to display multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [231]:
from shapely.geometry import Point
crs = {'init': 'epsg:4326'}

# Read data

In [232]:
DATA_ROOT = '../data/'
FIGURES_ROOT = '../figures/property'

READDIR_ROOT = os.path.join(DATA_ROOT, 'interim')
WRITEDIR_ROOT = os.path.join(DATA_ROOT, 'processed')

# Shapes to be considered
#  Changing this allows to match other shapes as well
readfile_tractshapes = os.path.join('../data/raw/shapefile/census-tracts/Census_2010_Tracts.shp')
gdf_tracts = gpd.read_file(readfile_tractshapes)

output_df = gdf_tracts[['geometry']].copy()
output_df['OBJECTID'] = gdf_tracts['OBJECTID']
output_df['area'] = gdf_tracts['Shape__Are']
output_df = output_df[['OBJECTID', 'area', 'geometry']]

In [233]:
# Dataframes to be included
prop_df_2018 = pd.read_csv(READDIR_ROOT + '/prop_df_2018.csv')
df_crime = pd.read_csv(READDIR_ROOT + '/crime/df_crime.csv')
df_schools = pd.read_csv(DATA_ROOT + '/raw/boston/public-k12-schools.csv')
df_streetlight = pd.read_csv(DATA_ROOT + 'raw/boston/streetlights.csv')
df_liquor = pd.read_csv(DATA_ROOT + '/processed/liquor_processed_final.csv')
df_violations = pd.read_csv(DATA_ROOT + 'raw/boston/property-violations.csv')

/Applications/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (16,61,64) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Applications/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (26,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Utils

In [234]:
def match_shapes(df, lat_label='LATITUDE', lon_label='LONGITUDE', obj_label='OBJECTID', useful_cols=None):
    '''Returns the df with additional columns with shape matches info'''
    
    points = [Point(coord[1], coord[0]) for coord in df[[lat_label, lon_label]].values]
    gdf_prop_2018 = gpd.GeoDataFrame(df, crs=crs, geometry=points)
    m = gpd.sjoin(gdf_prop_2018, gdf_tracts, how='left',op="within")
    print(m.columns)
    print("Matched", m[obj_label].value_counts().sum(), "out of", df.shape[0])
    
    if useful_cols:
        return m[[obj_label] + useful_cols]
    return m

In [235]:
traits = range(1, len(gdf_tracts) + 1)

def agg(df, f, col_label=None, id_label='OBJECTID'):
    '''Returns a new column with one row per tract, obtained by
    applying the function f to pd.Series obtained by taking rows with given OBJECTID
    '''
    
    l = []
    for t in traits:
        if col_label:
            my_df = (df[df[id_label] == t])[col_label]
        else:
            my_df = (df[df[id_label] == t])
            
        if isinstance(f, str):
            val = my_df.__getattribute__(f)()
        else:
            val = f(my_df)
        l.append(val)
    return pd.Series(l)

# Process data

## Property

In [236]:
# Sample usage: build a new df out of a dataset by matching shapes
#  optionally, select which columns will be useful to reduce runtime
useful = ['OWN_OCC', 'AV_LAND', 'AV_BLDG', 'GROSS_TAX', 
          'LAND_SF', 'YR_BUILT', 'LIVING_AREA', 'NUM_FLOORS']

new_prop = match_shapes(prop_df_2018, useful_cols=useful)

Index(['Unnamed: 0', 'PID', 'CM_ID', 'GIS_ID', 'ST_NUM', 'ST_NAME',
       'ST_NAME_SUF', 'UNIT_NUM', 'ZIPCODE', 'PTYPE',
       ...
       'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10', 'INTPTLAT10',
       'INTPTLON10', 'Shape_STAr', 'Shape_STLe', 'Shape__Are', 'Shape__Len'],
      dtype='object', length=105)
Matched 169648 out of 172841


In [237]:
# Applies the function given as a second argument to agg to
#  pd.Series object formed by selecting items in a tract separately

output_df['prop_count'] = agg(new_prop, len) # Counts number of items
output_df['prop_tax_mean'] = agg(new_prop, 'mean', col_label='GROSS_TAX') # Counts number of items
output_df['prop_tax_std'] = agg(new_prop, 'std', col_label='GROSS_TAX')
output_df['prop_living_area_mean'] = agg(new_prop, 'mean', col_label='LIVING_AREA')
output_df['prop_floor_mean'] = agg(new_prop, 'mean', col_label='NUM_FLOORS')
output_df['prop_land_mean'] = agg(new_prop, 'mean', col_label='LAND_SF')
output_df['prop_land_value_pct'] = agg(new_prop, 'sum', col_label='AV_LAND') /\
    (agg(new_prop, 'sum', col_label='AV_BLDG') + agg(new_prop, 'sum', col_label='AV_LAND'))

def pct_occ(s: pd.Series):
    '''Return the percentage of own occupancy'''
    try:
        return s.value_counts()['Y'] / len(s)
    except:
        return 0 # no own occupancy
output_df['prop_pct_own_occ'] = agg(new_prop, pct_occ, col_label='OWN_OCC')

def avg_year(s: pd.Series):
    return s.replace(0, s.median()).mean()
output_df['prop_avg_year_built'] = agg(new_prop, avg_year, col_label='YR_BUILT')


## Schools

In [238]:
new_schools = match_shapes(df_schools, lat_label='Y', lon_label='X', obj_label='OBJECTID_right')
output_df['school_count'] = agg(new_schools, len, id_label='OBJECTID_right') # Counts number of items


Index(['X', 'Y', 'OBJECTID_1', 'OBJECTID_left', 'BLDG_ID', 'BLDG_NAME',
       'ADDRESS', 'CITY', 'ZIPCODE', 'CSP_SCH_ID', 'SCH_ID', 'SCH_NAME',
       'SCH_LABEL', 'SCH_TYPE', 'SHARED', 'COMPLEX', 'POINT_X', 'POINT_Y',
       'geometry', 'index_right', 'FID', 'OBJECTID_right', 'STATEFP10',
       'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'MTFCC10',
       'FUNCSTAT10', 'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10',
       'Shape_STAr', 'Shape_STLe', 'Shape__Are', 'Shape__Len'],
      dtype='object')
Matched 131 out of 131


## Streetlight

In [239]:
new_streetlight = match_shapes(df_streetlight, lat_label='Lat', lon_label='Long', obj_label='OBJECTID_left')
output_df['streetlight_count'] = agg(new_streetlight, len, id_label='OBJECTID_left') # Counts number of items


Index(['the_geom', 'OBJECTID_left', 'TYPE', 'Lat', 'Long', 'geometry',
       'index_right', 'FID', 'OBJECTID_right', 'STATEFP10', 'COUNTYFP10',
       'TRACTCE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'MTFCC10', 'FUNCSTAT10',
       'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', 'Shape_STAr',
       'Shape_STLe', 'Shape__Are', 'Shape__Len'],
      dtype='object')
Matched 74065 out of 74065


## Liquor

In [240]:
new_liquor = match_shapes(df_liquor, lat_label='Latitude', lon_label='Longitude')
output_df['liquor_count'] = agg(new_liquor, len) # Counts number of items


Index(['Unnamed: 0', 'Unnamed: 0.1', 'LICENSENO', 'LICSTATUS', 'LICCAT',
       'LICCATDESC', 'Address_Complete', 'Location_Complete', 'Latitude',
       'Longitude', 'geometry', 'index_right', 'FID', 'OBJECTID', 'STATEFP10',
       'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'MTFCC10',
       'FUNCSTAT10', 'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10',
       'Shape_STAr', 'Shape_STLe', 'Shape__Are', 'Shape__Len'],
      dtype='object')
Matched 1048 out of 1103


## Violations

In [241]:
new_violations = match_shapes(df_violations, lat_label='latitude', lon_label='longitude')
output_df['violation_count'] = agg(new_violations, len) # Counts number of items


Index(['ticket_no', 'status_dttm', 'status', 'code', 'value', 'description',
       'stno', 'sthigh', 'street', 'suffix', 'city', 'state', 'zip', 'sam_id',
       'latitude', 'longitude', 'location', 'geometry', 'index_right', 'FID',
       'OBJECTID', 'STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'NAME10',
       'NAMELSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10',
       'INTPTLAT10', 'INTPTLON10', 'Shape_STAr', 'Shape_STLe', 'Shape__Are',
       'Shape__Len'],
      dtype='object')
Matched 18605 out of 18607


## Crime

In [252]:
# For now, just record the total number of crimes
#   More y-s to be added upon discussion

output_df['crime_count'] = agg(df_crime, len, id_label='OBJECTID_censustract') # Counts number of items


# Write to file

In [254]:
output_df.to_csv(WRITEDIR_ROOT + '/aggregate.csv')